   # Invokable Endpoints
   
   This example demonstrates how to quickly convert an invokable into a REST API
   endpoint via fastapi and the enact.fastapi integration package.

In [1]:
import asyncio
import uvicorn
import dataclasses
import fastapi
import random

import enact
import enact.fastapi

@enact.register
@dataclasses.dataclass
class MyResource(enact.Resource):
  x: int
  y: float

@enact.typed_invokable(input_type=MyResource, output_type=MyResource)
class MyInvokable(enact.Invokable):
  """Generates a resource with random values."""
  def call(self, max: MyResource):
    return MyResource(random.randint(0, max.x), random.random() * max.y)
    
app = fastapi.FastAPI()
my_invokable = MyInvokable()
store = enact.Store()
enact.fastapi.post(app, '/my_invokable/', my_invokable, store)


Deploy app using uvicorn:

In [2]:
config = uvicorn.Config(app)
server = uvicorn.Server(config)

# run server in background
serving_task = asyncio.create_task(server.serve())
await asyncio.sleep(1.0)  # Wait a second for startup

INFO:     Started server process [187992]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Access the server using a get requests:

In [3]:
import aiohttp

address = 'http://127.0.0.1:8000/my_invokable/'
async with aiohttp.ClientSession() as session:
  async with session.post(address, json={'x': 5, 'y': 3.0}) as resp:
    print(await resp.json())

INFO:     127.0.0.1:41430 - "POST /my_invokable/ HTTP/1.1" 200 OK
{'x': 2, 'y': 1.3912332354608665}


Shut down the server:

In [4]:
serving_task.cancel()
try:
  await serving_task
except asyncio.CancelledError:
  pass
await server.shutdown()

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
